<a href="https://colab.research.google.com/github/premswaroopmusti/Prophaze_tasks/blob/main/Task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Inferences that i will be finding out.**
**Requests per Minute:**

Calculate the frequency of requests occurring in each minute using the provided timestamps.

**IP Analysis:**

Count the unique IP addresses to determine the number of unique visitors.
Identify patterns in IP addresses, such as repeated visits from the same IP or IP ranges.

**Request Analysis:**

Count the frequency of different types of requests (e.g., resource paths) to identify popular resources.
Analyze the status codes to understand the distribution of successful and unsuccessful requests.

**Referrer Analysis:**

Count the frequency of different referrers to identify sources of traffic.
Determine if there is a significant portion of requests with no referrer.

**User-Agent Analysis:**

Count the frequency of different user agents to understand the distribution of devices and browsers.
Identify the most common user agents and their associated devices/browsers.

**Temporal Patterns:**

Analyze the timestamp data to identify peak traffic times and periods of low activity.
Calculate average request rates per hour.

**Geolocation:**

Use IP geolocation services to determine the geographical distribution of the requests

### **Importing the Dependencies**

In [ ]:
import pandas as pd
from dateutil import parser
from collections import Counter
from datetime import datetime, date
import numpy as np
from collections import OrderedDict
import re
import requests
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive

### **Data Collection & Pre-Processing**

In [ ]:
drive.mount('/content/drive')

# loading the data from csv file to a pandas Dataframe
data = pd.read_csv("/content/drive/MyDrive/Dataset/123dataset.csv");

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129400 entries, 0 to 129399
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Unnamed: 0       129400 non-null  int64 
 1   request          129400 non-null  object
 2   remote_addr      129400 non-null  object
 3   @timestamp       129400 non-null  object
 4   http_referrer    129400 non-null  object
 5   http_user_agent  129400 non-null  object
 6   status           129400 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 6.9+ MB


In [ ]:
timevalue_list = list(data['@timestamp'])
request_list = list(data['request'])
ip_address = list(data['remote_addr'])
reffer_list = list(data['http_referrer'])
status_list = list(data['status'])
user_agent_list = list(data['http_user_agent'])

**Requests per Minute:**

Calculate the frequency of requests occurring in each minute using the provided timestamps.

In [ ]:
 # Request Occur in Every Minute
total_request_count = 0
requests_per_minute = Counter()
for timestamp_str in timevalue_list:
    timestamp_str = timestamp_str.split('+')[0]
    timestamp_str = timestamp_str.split('.')[0]
    timestamp = datetime.strptime(timestamp_str, "%Y-%m-%d %H:%M:%S")
    minute = timestamp.strftime("%Y-%m-%d %H:%M")
    requests_per_minute[minute] += 1
for minute, count in requests_per_minute.items():
    # print(f"Minute: {minute}, Requests: {count}")
    total_request_count += count
requests_per_minute = sorted(requests_per_minute)
total_min_count = 5*24*60
request_per_min = total_request_count//total_min_count
print("Request_per_min : ",request_per_min)

Request_per_min :  17


**IP Analysis:**


In [ ]:
# do all IPs perform the same request
Ip_with_request_type_count = {}

for i in range(len(ip_address)):
  if ip_address[i] in Ip_with_request_type_count.keys():
    if "get" in str(request_list[i]).lower():
      if "GET" in Ip_with_request_type_count[ip_address[i]].keys():
        Ip_with_request_type_count[ip_address[i]]['GET'] += 1
      else:
        Ip_with_request_type_count[ip_address[i]]['GET'] = 1
    elif "post" in str(request_list[i]).lower():
      if "POST" in Ip_with_request_type_count[ip_address[i]].keys():
        Ip_with_request_type_count[ip_address[i]]['POST'] += 1
      else:
        Ip_with_request_type_count[ip_address[i]]['POST'] = 1
    else:
      if "HEAD" in Ip_with_request_type_count[ip_address[i]].keys():
        Ip_with_request_type_count[ip_address[i]]['HEAD'] += 1
      else:
        Ip_with_request_type_count[ip_address[i]]['HEAD'] = 1
  else:
    Ip_with_request_type_count[ip_address[i]] = {}
    if "get" in str(request_list[i]).lower():
      if "GET" in Ip_with_request_type_count[ip_address[i]].keys():
        Ip_with_request_type_count[ip_address[i]]['GET'] += 1
      else:
        Ip_with_request_type_count[ip_address[i]]['GET'] = 1
    elif "post" in str(request_list[i]).lower():
      if "POST" in Ip_with_request_type_count[ip_address[i]].keys():
        Ip_with_request_type_count[ip_address[i]]['POST'] += 1
      else:
        Ip_with_request_type_count[ip_address[i]]['POST'] = 1
    else:
      if "HEAD" in Ip_with_request_type_count[ip_address[i]].keys():
        Ip_with_request_type_count[ip_address[i]]['HEAD'] += 1
      else:
        Ip_with_request_type_count[ip_address[i]]['HEAD'] = 1

Ip_with_request_type_count

{'65.20.79.142': {'GET': 22292, 'POST': 4920, 'HEAD': 6082},
 '65.20.74.204': {'GET': 44741, 'POST': 8063, 'HEAD': 12111},
 '65.20.67.138': {'GET': 21092, 'POST': 4063, 'HEAD': 6036}}

In [29]:
import numpy as np

# Create a dictionary to store the count of referrers for each IP address
ip_source_count = {}

# Loop through the IP addresses and referrers
for i in range(len(ip_address)):
    ip = ip_address[i]
    referrer = reffer_list[i]

    if ip in ip_source_count:
        if referrer in ip_source_count[ip]:
            ip_source_count[ip][referrer] += 1
        else:
            ip_source_count[ip][referrer] = 1
    else:
        ip_source_count[ip] = {referrer: 1}

# Create a dictionary to store the top 10 referrers for each IP address
top_10_referrers = {}

# Loop through the IP addresses and their referrer counts
for ip, referrer_count in ip_source_count.items():
    # Sort referrers by count in descending order and get the top 10
    top_referrers = sorted(referrer_count.items(), key=lambda x: x[1], reverse=True)[:10]
    top_10_referrers[ip] = top_referrers

# Print the top 10 referrers for each IP address
for ip, referrers in top_10_referrers.items():
    print(f"IP Address: {ip}")
    for referrer, count in referrers:
        print(f"  Referrer: {referrer}, Count: {count}")
    print()


IP Address: 65.20.79.142
  Referrer: None, Count: 16808
  Referrer: https://prophaze.com/, Count: 2691
  Referrer: https://prophaze.com/cve/best-practical-request-tracker-up-to-4-2-16-4-4-4-5-0-1-auth-pm-timing-discrepancy/, Count: 1320
  Referrer: https://prophaze.com/web-application-firewall/free-ssl-certificate-providers/, Count: 550
  Referrer: https://prophaze.com, Count: 351
  Referrer: https://prophaze.com:443//wp-login.php, Count: 336
  Referrer: https://prophaze.com/learning/components-with-known-vulnerabilities/, Count: 263
  Referrer: https://prophaze.com/cve/cve-2023-39213/, Count: 240
  Referrer: https://prophaze.com/cve/monstra-cms-3-0-4-page-feature-admin-index-php-cross-site-scripting/, Count: 216
  Referrer: https://prophaze.com/solutions/aws-waf/, Count: 182

IP Address: 65.20.74.204
  Referrer: None, Count: 31808
  Referrer: https://prophaze.com/, Count: 5660
  Referrer: https://prophaze.com/cve/best-practical-request-tracker-up-to-4-2-16-4-4-4-5-0-1-auth-pm-timing-d

**Request Analysis:**


Analyze the status codes to understand the distribution of successful and unsuccessful requests.

In [ ]:
status_information = {
    "successful": 0,
    "unsuccessful" : 0
}

for i in status_list:
  if i >= 200 and i < 300:
    status_information['successful'] += 1
  else:
    status_information['unsuccessful'] += 1

status_information

{'successful': 94265, 'unsuccessful': 35135}

**Referrer Analysis:**


Count the frequency of different referrers to identify sources of traffic.

Determine if there is a significant portion of requests with no referrer.

In [20]:
reffer_count = {}
threshold = 200
popular_reffer_count = []
for i in reffer_list:
    if i in reffer_count.keys():
        reffer_count[i] += 1
        if reffer_count[i] == threshold:
            popular_reffer_count.append(i)
    else:
        reffer_count[i] = 1

def sort_dict_by_value(d, reverse=False):
    return dict(sorted(d.items(), key=lambda x: x[1], reverse=reverse))

reffer_count = sort_dict_by_value(reffer_count, True)

# Get the top 10 referrers
top_10_reffer = list(reffer_count.keys())[:10]

# Print the top 10 referrers along with their counts
for referrer in top_10_reffer:
    count = reffer_count[referrer]
    print(f"Referrer: {referrer}, Count: {count}")


Referrer: None, Count: 64250
Referrer: https://prophaze.com/, Count: 11171
Referrer: https://prophaze.com/cve/best-practical-request-tracker-up-to-4-2-16-4-4-4-5-0-1-auth-pm-timing-discrepancy/, Count: 5494
Referrer: https://prophaze.com/web-application-firewall/free-ssl-certificate-providers/, Count: 1946
Referrer: https://prophaze.com, Count: 1452
Referrer: https://prophaze.com/learning/components-with-known-vulnerabilities/, Count: 937
Referrer: https://prophaze.com/cve/monstra-cms-3-0-4-page-feature-admin-index-php-cross-site-scripting/, Count: 874
Referrer: https://prophaze.com/solutions/aws-waf/, Count: 774
Referrer: https://prophaze.com/cve/cve-2023-39213/, Count: 757
Referrer: https://prophaze.com/free-waf/, Count: 754



**User-Agent Analysis:**

Count the frequency of different user agents to understand the distribution of devices and browsers.

In [23]:
unique_devices = set()
unique_browsers = set()

for user_agent in user_agent_list:
    # Extract device information
    device_match = re.search(r'\(([^;]+);[^\)]+\)', user_agent)
    if device_match:
        unique_devices.add(device_match.group(1).strip())

    # Extract browser information
    browser_match = re.search(r' ([a-zA-Z]+)/\d+', user_agent)
    if browser_match:
        unique_browsers.add(browser_match.group(1))

print("Unique Devices:", unique_devices)
print()
print("Unique Browsers:", unique_browsers)

Unique Devices: {'Windows NT 6.0', 'Linux x86_64', 'Windows NT 6.1', 'Android 4.0.4', 'Macintosh', 'Windows NT 10.0\\', 'Android 13', 'Windows NT 10.0', 'Windows NT 10.0.0', 'Windows NT 6.3', 'Windows NT 5.1', 'Windows NT 7.5', 'Android 11', 'iPhone', 'KHTML, like Gecko', 'Windows NT 8_0_1', 'Windows', '/global', 'X11', 'iPad', 'webOS/1.4.5', 'iPhone Simulator', 'iPod', 'Windows NT 8.1', 'https://notion.so', 'compatible', '+https://awario.com/bots.html', 'Windows NT 6.2', 'iPod touch', 'Linux'}

Unique Browsers: {'CensysInspect', 'zgrab', 'DotBot', 'InternetMeasurement', 'YandexImages', 'spider', 'YaK', 'CFNetwork', 'Cincraw', 'linkdexbot', 'Trident', 'BLEXBot', 'Konturbot', 'Baiduspider', 'Daum', 'Chrome', 'Spider', 'NetcraftSurveyAgent', 'Googlebot', 'AwarioBot', 'MojeekBot', 'Gecko', 'DataForSeoBot', 'UptimeRobot', 'adidxbot', 'aiohttp', 'YandexRenderResourcesBot', 'Firefox', 'Semanticbot', 'Fetcher', 'BOLT', 'SerpReputationManagementAgent', 'SeznamBot', 'YandexFavicons', 'SiteAudit

Identify the most common user agents and their associated devices/browsers.

In [30]:
device_counter = Counter()
browser_counter = Counter()

for user_agent in user_agent_list:
    # Extract device information
    device_match = re.search(r'\(([^;]+);[^\)]+\)', user_agent)
    if device_match:
        device = device_match.group(1).strip()
        device_counter[device] += 1

    # Extract browser information
    browser_match = re.search(r' ([a-zA-Z]+)/\d+', user_agent)
    if browser_match:
        browser = browser_match.group(1)
        browser_counter[browser] += 1

# Get top frequent devices and browsers
top_devices = device_counter.most_common(10)
top_browsers = browser_counter.most_common(10)

print("Top Frequent Devices:")
print()
for device, count in top_devices:
    print(f"{device}: {count}")

print("\nTop Frequent Browsers:")
print()
for browser, count in top_browsers:
    print(f"{browser}: {count}")


Top Frequent Devices:

Windows NT 10.0: 46796
compatible: 16301
Linux: 10806
X11: 9153
Macintosh: 6982
iPhone: 2629
KHTML, like Gecko: 2388
Windows NT 6.1: 1710
+https://awario.com/bots.html: 1253
Windows NT 6.3: 733

Top Frequent Browsers:

AppleWebKit: 69945
Gecko: 12600
VelenPublicWebCrawler: 4868
UptimeRobot: 2894
DotBot: 2624
YandexBot: 1309
YandexRenderResourcesBot: 1071
Googlebot: 806
AhrefsBot: 520
Trident: 271


**Temporal Patterns:**

Analyze the timestamp data to identify peak traffic times and periods of low activity.

 Calculate average request rates per hour

In [31]:
# Request Occur in Every Hour
total_request_count = 0
requests_per_hour = Counter()
for timestamp_str in timevalue_list:
    timestamp_str = timestamp_str.split('+')[0]
    timestamp_str = timestamp_str.split('.')[0]
    timestamp = datetime.strptime(timestamp_str, "%Y-%m-%d %H:%M:%S")
    hour = timestamp.strftime("%Y-%m-%d %H")
    requests_per_hour[hour] += 1
requests_per_hour = sorted(requests_per_hour.items(), key=lambda x: x[1], reverse=True)
for hour, count in requests_per_hour:
    # print(f"Hour: {hour}, Requests: {count}")
    total_request_count += count
# requests_per_hour = sorted(requests_per_hour)
# requests_per_hour
total_hour_count = 5*24
request_per_hour = total_request_count//total_hour_count
print("Request_per_hour : ",request_per_hour)
print()
# Identify peak and low traffic hours
peak_hours = requests_per_hour[:5]  # Get the top 5 hours with the highest request counts
low_hours = requests_per_hour[-5:]  # Get the 5 hours with the lowest request counts

print("Peak Traffic Hours:")
print()
for hour, count in peak_hours:
    print(f"Hour: {hour}, Requests: {count}")

print("\nLow Traffic Hours:")
print()
for hour, count in low_hours:
    print(f"Hour: {hour}, Requests: {count}")


Request_per_hour :  1078

Peak Traffic Hours:

Hour: 2023-08-11 04, Requests: 21971
Hour: 2023-08-15 13, Requests: 2668
Hour: 2023-08-11 17, Requests: 2227
Hour: 2023-08-14 13, Requests: 2137
Hour: 2023-08-13 16, Requests: 1700

Low Traffic Hours:

Hour: 2023-08-12 03, Requests: 379
Hour: 2023-08-12 21, Requests: 376
Hour: 2023-08-15 02, Requests: 354
Hour: 2023-08-15 09, Requests: 331
Hour: 2023-08-14 02, Requests: 273


**Geolocation:**

Use IP geolocation services to determine the geographical distribution of the requests

In [ ]:
# IP address to geolocate
ip_address_set = set(ip_address)
ip_info_obj ={}

for i in ip_address_set:
  # Make a GET request to the ipinfo.io API
  response = requests.get(f"https://ipinfo.io/{i}/json")

  # Check if the request was successful
  if response.status_code == 200:
      # Parse the JSON response
      ip_info = response.json()
      ip_info_obj[i]= {}
      # Extract relevant information (e.g., city, region, country)
      ip_info_obj[i]['city'] = ip_info.get("city", "N/A")
      ip_info_obj[i]['region'] = ip_info.get("region", "N/A")
      ip_info_obj[i]['country'] = ip_info.get("country", "N/A")
      ip_info_obj[i]['org'] = ip_info.get("org", "N/A")
      # Print the geolocation information
      # print(f"IP Address: {ip_address}")
      # print(f"City: {city}")
      # print(f"Region: {region}")
      # print(f"Country: {country}")
      # print(f"Organization: {org}")
  else:
      print(f"Failed to retrieve geolocation data for IP: {i}")

ip_info_obj

{'65.20.67.138': {'city': 'Mumbai',
  'region': 'Maharashtra',
  'country': 'IN',
  'org': 'AS20473 The Constant Company, LLC'},
 '65.20.79.142': {'city': 'Mumbai',
  'region': 'Maharashtra',
  'country': 'IN',
  'org': 'AS20473 The Constant Company, LLC'},
 '65.20.74.204': {'city': 'Mumbai',
  'region': 'Maharashtra',
  'country': 'IN',
  'org': 'AS20473 The Constant Company, LLC'}}